In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Dikson,RU,73.5069,80.5464,-5.46,98,100,15.77,overcast clouds
1,1,Saint-Pierre,RE,-21.3393,55.4781,76.91,87,75,13.80,broken clouds
2,2,Ushuaia,AR,-54.8000,-68.3000,38.86,87,100,17.27,moderate rain
3,3,Garden City,US,42.3256,-83.3310,37.98,65,0,8.05,clear sky
4,4,Faanui,PF,-16.4833,-151.7500,81.99,69,100,9.69,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 45
What is the maximum temperature you would like for your trip? 78


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Saint-Pierre,RE,-21.3393,55.4781,76.91,87,75,13.80,broken clouds
6,6,Ridgecrest,US,35.6225,-117.6709,75.06,16,0,24.16,clear sky
10,10,Hermanus,ZA,-34.4187,19.2345,53.80,67,96,12.57,light rain
17,17,Mataura,NZ,-46.1927,168.8643,65.48,49,47,4.41,scattered clouds
20,20,Saint-Philippe,RE,-21.3585,55.7679,76.21,87,100,11.86,moderate rain
24,24,Lambarene,GA,-0.7001,10.2406,75.78,94,94,1.25,light rain
27,27,East London,ZA,-33.0153,27.9116,68.07,94,100,13.89,moderate rain
28,28,Mar Del Plata,AR,-38.0023,-57.5575,66.09,87,4,11.99,clear sky
31,31,Bluff,NZ,-46.6000,168.3333,65.80,74,32,5.82,scattered clouds
32,32,Lima,PE,-12.0432,-77.0282,72.39,73,100,7.27,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                440
City                   440
Country                434
Lat                    440
Lng                    440
Max Temp               440
Humidity               440
Cloudiness             440
Wind Speed             440
Current Description    440
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna(inplace = True)
preferred_cities_df.count()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


City_ID                434
City                   434
Country                434
Lat                    434
Lng                    434
Max Temp               434
Humidity               434
Cloudiness             434
Wind Speed             434
Current Description    434
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Saint-Pierre,RE,76.91,broken clouds,-21.3393,55.4781,
6,Ridgecrest,US,75.06,clear sky,35.6225,-117.6709,
10,Hermanus,ZA,53.80,light rain,-34.4187,19.2345,
17,Mataura,NZ,65.48,scattered clouds,-46.1927,168.8643,
20,Saint-Philippe,RE,76.21,moderate rain,-21.3585,55.7679,
24,Lambarene,GA,75.78,light rain,-0.7001,10.2406,
27,East London,ZA,68.07,moderate rain,-33.0153,27.9116,
28,Mar Del Plata,AR,66.09,clear sky,-38.0023,-57.5575,
31,Bluff,NZ,65.80,scattered clouds,-46.6000,168.3333,
32,Lima,PE,72.39,overcast clouds,-12.0432,-77.0282,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"  
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")   
        
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna(inplace = True)

In [10]:
# 8a. Create the output File (CSV)
#output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
#hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))